<br>

<p style="text-align:center; font-weight:bold; font-size:17px;">Normal & Log-normal distribution</p>

<p style="text-align:center; font-size:14px;">In this notebook, we want to study and compare the relationship between normal and log-normal distribution.</p>

<p style="text-align:center; font-size:14px;">If a random variable X follows a normal distribution, then the exponential of that variable, exp(X), follows a log-normal distribution.</p>

<p style="text-align:center; font-size:14px;">Conversely, if a random variable Y follows a log-normal distribution, then the natural logarithm of that variable, ln(Y), follows a normal distribution.</p>


In [1]:
#-------
# IMPORT BUILT-IN LIBRAIRIES
#-------
import numpy as np
import scipy.stats as stats
from scipy.integrate import quad

#-------
# DEFINE PROBABILITY DENSITY FUNCTION
#-------
def standard_normal_pdf(x, mu, sigma):
    exponent = -((x - mu) ** 2) / (2 * sigma ** 2)
    coefficient = 1 / (np.sqrt(2 * np.pi * sigma ** 2))
    pdf = coefficient * np.exp(exponent)
    return pdf

def log_normal_pdf(x, mu, sigma):
    exponent = -((np.log(x) - mu) ** 2) / (2 * sigma ** 2)
    coefficient = 1 / (x * sigma * np.sqrt(2 * np.pi))
    pdf = coefficient * np.exp(exponent)
    return pdf

<p style="text-align:center; font-weight:bold; font-size:15px;">Example 1: Probability</p>


In [6]:
#-------
# INPUT
#-------
mu = 0
std = 0.2
normal_lower_bound = -std*10
normal_upper_bound = std*10
log_lower_bound = np.exp(normal_lower_bound)
log_upper_bound = np.exp(normal_upper_bound)

#-------
# COMPUTE MEAN AND PROBABILITY
#-------
normal_proba = quad(standard_normal_pdf, normal_lower_bound, normal_upper_bound, args=(mu, std))[0]
log_proba = quad(log_normal_pdf, log_lower_bound, log_upper_bound, args=(mu, std))[0]
print(f'Using {normal_lower_bound}/{normal_upper_bound} for normal distribution and {log_lower_bound:.2F}/{log_upper_bound:.2F} for log-normal distribution')
print(f'Standard normal probability {normal_proba}')
print(f'Log normal probability {log_proba}')

Using -2.0/2.0 for normal distribution and 0.14/7.39 for log-normal distribution
Standard normal probability 1.0
Log normal probability 1.0000000000000002


<p style="text-align:center; font-weight:bold; font-size:15px;">Example 2: Option Pricing</p>


In [3]:
#-------
# PARAMETERS
#-------
D = 252
S0 = 100
K = 100
T = D/252
r = 0.0
sig = 0.20

#-------
# PRICING
#-------
e_ret = np.log(S0) + (r - 0.5 * sig**2) * T
vol = sig * np.sqrt(T)

def log_normal_integration(S, strike, e_ret, vol, payoff):
    intrinsec_value = max(0, (S - strike)) if payoff == 'call' else max(0, (strike - S))
    return intrinsec_value * log_normal_pdf(x=S, mu=e_ret, sigma=vol)

def standard_normal_integration(S, strike, e_ret, vol, payoff):
    adj_intrinsec_value = max(0, (np.exp(S) - strike)) if payoff == 'call' else max(0, (strike - np.exp(S)))
    return adj_intrinsec_value * standard_normal_pdf(x=S, mu=e_ret, sigma=vol)

def standard_normal_integration_scipy(S, strike, e_ret, vol, payoff):
    adj_intrinsec_value = max(0, (np.exp(S) - strike)) if payoff == 'call' else max(0, (strike - np.exp(S)))
    return adj_intrinsec_value * stats.norm.pdf(x=S, loc=e_ret, scale=vol)

def standard_normal_integration_proba(S, strike, e_ret, vol):
    p = 1 if max(0, (np.exp(S) - strike)) > 0 else 0
    return p * standard_normal_pdf(x=S, mu=e_ret, sigma=vol)

call = quad(log_normal_integration, K, np.inf, args=(K, e_ret, vol, "call"))[0] * np.exp(-r * T)
put = quad(log_normal_integration, 0, K, args=(K, e_ret, vol, "put"))[0] * np.exp(-r * T)
print(f"With Log-normal integration: Call price={call}  Put price={put}")

call = quad(standard_normal_integration, np.log(K), 10, args=(K, e_ret, vol, "call"))[0] * np.exp(-r * T)
put = quad(standard_normal_integration, -10, np.log(K), args=(K, e_ret, vol, "put"))[0] * np.exp(-r * T)
print(f"With Standard normal integration: Call price={call}  Put price={put}")

call = quad(standard_normal_integration_proba, -10, 10, args=(K, e_ret, vol))[0] * np.exp(-r * T)
print(f"With Standard normal integration: Call probability={call}")

call = quad(standard_normal_integration_scipy, np.log(K), 10, args=(K, e_ret, vol, "call"))[0] * np.exp(-r * T)
put = quad(standard_normal_integration_scipy, -10, np.log(K), args=(K, e_ret, vol, "put"))[0] * np.exp(-r * T)
print(f"With Standard normal integration and scipy: Call price={call}  Put price={put}")

With Log-normal integration: Call price=7.965567455405844  Put price=7.9655674554057585
With Standard normal integration: Call price=7.965567455405843  Put price=7.9655674554057585
With Standard normal integration: Call probability=0.46017216064068267
With Standard normal integration and scipy: Call price=7.96556745540584  Put price=7.965567455405757
